# Dump results of multiple snapshots

- Flow

In [ ]:
import glob
import os

loss = 'lossintra'
cue = 'flow'
# loss = 'lossinter'
# cue = 'flow'

prototxts = glob.glob('prototxts/*.prototxt')
protobins = sorted(glob.glob('snapshots/*.caffemodel'))

def is_model(seed, filename, cue='rgb', loss='lossintra'):
    return ('rng' in filename and seed in filename and
            loss in filename and cue in filename)

def is_prototxt(filename, cue='rgb', loss='lossintra'):
    return ('deploy' in filename and 'rng' in filename and
            loss in filename and cue in filename)

for i in prototxts:
    if is_prototxt(i, cue=cue, loss=loss):
        seed = i.split('_rng')[1].split('_')[0]
        
        protobins_i = [j for j in protobins
                       if is_model(seed, j, cue=cue, loss=loss)]
        if len(protobins_i) < 3:
            continue
        j = os.path.basename(protobins_i[0]).split('_iter_')[0]        
        
        !bash assess_flow.sh $i $j

- RGB

In [ ]:
import glob
import os

loss = 'lossintra'
cue = 'rgb'
# loss = 'lossinter'
# cue = 'rgb'

prototxts = glob.glob('prototxts/*.prototxt')
protobins = sorted(glob.glob('snapshots/*.caffemodel'))

def is_model(seed, filename, cue='rgb', loss='lossintra'):
    return ('rng' in filename and seed in filename and
            loss in filename and cue in filename)

def is_prototxt(filename, cue='rgb', loss='lossintra'):
    return ('deploy' in filename and 'rng' in filename and
            loss in filename and cue in filename)

for i in prototxts:
    if is_prototxt(i, cue=cue, loss=loss):
        seed = i.split('_rng')[1].split('_')[0]
        
        protobins_i = [j for j in protobins
                       if is_model(seed, j, cue=cue, loss=loss)]
        if len(protobins_i) < 3:
            continue
        
        j = os.path.basename(protobins_i[0]).split('_iter_')[0]        
        !bash assess_rgb.sh $i $j

cross-check results and prototxts

In [ ]:
import glob
print(len(glob.glob('prototxts/deploy_*_rgb_*_lossinter_*.prototxt')))
print(len(glob.glob('results/clip_retrieval_rgb_*_lossinter_*_val.p')))

# Get best seed and iteration

In [2]:
import glob
import pickle
import numpy as np

loss = 'lossintra'
cue = 'rgb'
ind_metric_for_tie = 1


def best_along_multiple_columns(x, ind_col_if_tied=1):
    votes = np.argmax(x, axis=0)
    candidates, counts = np.unique(votes, return_counts=True)
    if len(candidates) == x.shape[1]:
        return np.argmax(x[:, ind_col_if_tied])
    else:
        return candidates[np.argsort(counts)[-1]]

for (cue, loss) in [('rgb', 'lossintra'),
                    ('flow', 'lossintra'),
                    ('rgb', 'lossinter'),
                    ('flow', 'lossinter')]:
    results = glob.glob('results/*{}*rng*{}*.p'
                    .format(cue, loss))
    
    y = []
    seed = []
    best_iter = []
    for i in results:
        with open(i, 'rb') as f:
            data = pickle.load(f)

        # pick best iteration
        x = []
        iterations = []
        for j in data:
            if 'performance' in data[j]:
                iterations.append(j)
                x.append(data[j]['performance'])
        x_arr = np.array(x)
        ind = best_along_multiple_columns(
            x_arr, ind_col_if_tied=ind_metric_for_tie)
        y.append(x_arr[ind, :])
        best_iter.append(iterations[ind])
        seed_i = i.split('rng')[1].split('_')[0]
        seed.append(seed_i)

    print(cue, loss)
    # compute mean and std per metric
    y_arr = np.array(y)
    print('Avg: ', y_arr.mean(axis=0))
    print('Std: ', y_arr.std(axis=0))

    # pick best rng
    ind = best_along_multiple_columns(
        y_arr, ind_col_if_tied=ind_metric_for_tie)
    print('Best: ', y_arr[ind, :])
    print('seed: ', seed[ind], '; iter: ', best_iter[ind])
    print

('rgb', 'lossintra')
('Avg: ', array([0.2373445 , 0.74633971, 0.35395362]))
('Std: ', array([0.00319618, 0.00351935, 0.00217724]))
('Best: ', array([0.23588517, 0.75287081, 0.35795322]))
('seed: ', '3779497804', '; iter: ', 30000)

('flow', 'lossintra')
('Avg: ', array([0.26291866, 0.77370813, 0.38822568]))
('Std: ', array([0.005374  , 0.00326527, 0.00586131]))
('Best: ', array([0.27272727, 0.77129187, 0.39913477]))
('seed: ', '384112814', '; iter: ', 30000)

('rgb', 'lossinter')
('Avg: ', array([0.11411483, 0.35339713, 0.29859171]))
('Std: ', array([0.0091524 , 0.02353921, 0.01084511]))
('Best: ', array([0.13636364, 0.40167464, 0.31169192]))
('seed: ', '2227967492', '; iter: ', 10000)

('flow', 'lossinter')
('Avg: ', array([0.14449761, 0.41255981, 0.34805409]))
('Std: ', array([0.00566233, 0.01607337, 0.00480636]))
('Best: ', array([0.15526316, 0.43899522, 0.3463756 ]))
('seed: ', '2227967492', '; iter: ', 20000)



double check above procedure is right

In [ ]:
iteration = 30000
seed = 384112814
cue = 'rgb'
cue = 'flow'
loss = 'lossintra'
loss = 'lossinter'

filename = 'results/clip_retrieval_{}_hachiko_rng{}_feature_process_context_recurrent_embedding_lfTrue_dv0.3_dl0.0_nlv2_nlllstm_no_embed_edl1000-100_edv500-100_pmFalse_{}_lwInter1_val.p'
data = pickle.load(open(filename.format(cue, seed, loss), 'rb'))
data[iteration]['performance']

# Dump features

manual dump

In [3]:
import glob
rgb_seed, flow_seed = 2227967492, 2227967492
rgb_iter, flow_iter = 10000, 20000
loss = 'lossinter'

def get_file(wildcard):
    files = glob.glob(wildcard)
    assert len(files) == 1
    return files[0]

prototxt_fmt = 'prototxts/deploy_clip_retrieval_clip_retrieval_{}_hachiko_rng{}_feature_process_context_recurrent_embedding_lfTrue_dv0.3_dl0.0_nlv2_nlllstm_no_embed_edl1000-100_edv500-100_pmFalse_{}_*.prototxt'
protobin_fmt = 'snapshots/clip_retrieval_{}_hachiko_rng{}_feature_process_context_recurrent_embedding_lfTrue_dv0.3_dl0.0_nlv2_nlllstm_no_embed_edl1000-100_edv500-100_pmFalse_{}_*_iter_{}.caffemodel'
rgb_prototxt = get_file(prototxt_fmt.format('rgb', rgb_seed, loss))
flow_prototxt = get_file(prototxt_fmt.format('flow', flow_seed, loss))
rgb_protobin = get_file(protobin_fmt.format('rgb', rgb_seed, loss, rgb_iter))
flow_protobin = get_file(protobin_fmt.format('flow', flow_seed, loss, flow_iter))
!bash dump_features.sh $rgb_prototxt $flow_prototxt $rgb_protobin $flow_protobin

RGB model on val...
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Reading glove embedding
4170/4180Dumping features
Dumped results to: results/queries_val.hdf5
Dumped results to: results/corpus_val.hdf5
Flow model on val...
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Reading glove embedding
4170/4180Dumping features
Dumped results to: results/queries_val_flow.hdf5
Dumped results to: results/corpus_val_flow.hdf5
RGB model on test...
/usr/local/lib/python2.7/dist-packages/h5py/__